In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import torch
import torch.nn as nn

In [12]:
!mkdir /content/darknet

!unzip "/content/drive/MyDrive/yoloclothes.zip" -d "/content/darknet"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/darknet/data/images/6640.txt  
  inflating: /content/darknet/data/images/6641.jpg  
  inflating: /content/darknet/data/images/6641.txt  
  inflating: /content/darknet/data/images/6642.jpg  
  inflating: /content/darknet/data/images/6642.txt  
  inflating: /content/darknet/data/images/6643.jpg  
  inflating: /content/darknet/data/images/6643.txt  
  inflating: /content/darknet/data/images/6644.jpg  
  inflating: /content/darknet/data/images/6644.txt  
  inflating: /content/darknet/data/images/6645.jpg  
  inflating: /content/darknet/data/images/6645.txt  
  inflating: /content/darknet/data/images/6646.jpg  
  inflating: /content/darknet/data/images/6646.txt  
  inflating: /content/darknet/data/images/6647.jpg  
  inflating: /content/darknet/data/images/6647.txt  
  inflating: /content/darknet/data/images/6648.jpg  
  inflating: /content/darknet/data/images/6648.txt  
  inflating: /content/darknet/data/images/6649.jpg  
  inflatin

In [13]:

%cd /content/darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1' Makefile

!make
!chmod +x ./darknet


/content/darknet
sed: -e expression #1, char 17: unterminated `s' command
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentatio

In [19]:
!rm -rf /content/darknet/backup
!ln -s /content/drive/'MyDrive'/yolov3/backup/content/darknet




!sudo apt install dos2unix

!dos2unix ./data/list/v3train.txt
!dos2unix ./data/list/v3valid.txt
!dos2unix ./data/clothesdef.names
!dos2unix ./data/obj.data
!dos2unix ./cfg/yolov3_testing.cfg

ln: failed to create symbolic link './darknet': File exists
Reading package lists... Done
Building dependency tree       
Reading state information... Done
dos2unix is already the newest version (7.3.4-3).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
dos2unix: converting file ./data/list/v3train.txt to Unix format...
dos2unix: converting file ./data/list/v3valid.txt to Unix format...
dos2unix: converting file ./data/clothesdef.names to Unix format...
dos2unix: converting file ./data/obj.data to Unix format...
dos2unix: converting file ./cfg/yolov3_testing.cfg to Unix format...


In [15]:
def imShow(path): 
  import cv2 
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image, (3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [16]:
%cd /content/darknet

!wget !wget https://pjreddie.com/media/files/darknet53.conv.74

/content/darknet
--2021-12-03 12:22:32--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2021-12-03 12:22:32--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.2’

darknet53.conv.74.2 100%[===================>] 154.96M  21.3MB/s    in 8.1s    

2021-12-03 12:22:41 (19.1 MB/s) - ‘darknet53.conv.74.2’ saved [162482580/162482580]

FINISHED --2021-12-03 12:22:41--
Total wall clock time: 9.0s
Downloaded: 1 files, 155M in 8.1s (19.1 MB/s)


In [17]:
torch.cuda.empty_cache()

In [ ]:
%cd /content/darknet

!time ./darknet detector train data/obj.data cfg/yolov3_testing.cfg darknet53.conv.74  -dont_show  -map l tee train.log

!cp -r /content/drive/MyDrive/yolov3/backup

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 (next mAP calculation at 3315 iterations) 
 88: 556.058655, 695.235962 avg loss, 0.000000 rate, 1.202693 seconds, 704 images, 25.680995 hours left
Loaded: 0.000099 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.423595), count: 8, class_loss = 85.020287, iou_loss = 1.762199, total_loss = 86.782486 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 362.116791, iou_loss = 0.000000, total_loss = 362.116791 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 1182.746094, iou_loss = 0.000000, total_loss = 1182.746094 
 total_bbox = 724, rewritten_bbox = 0.000000 % 

 (next mAP calculation at 3315 iterations) 
 89: 543.881775, 680.100525 avg loss, 0.000000 rate, 1.197100 seconds, 712 images, 25.631715 hours left
Loaded: 0.000135 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls

# 새 섹션